
<br>
===========================================<br>
Spectral clustering for image segmentation<br>
===========================================<br>
In this example, an image with connected circles is generated and<br>
spectral clustering is used to separate the circles.<br>
In these settings, the :ref:`spectral_clustering` approach solves the problem<br>
know as 'normalized graph cuts': the image is seen as a graph of<br>
connected voxels, and the spectral clustering algorithm amounts to<br>
choosing graph cuts defining regions while minimizing the ratio of the<br>
gradient along the cut, and the volume of the region.<br>
As the algorithm tries to balance the volume (ie balance the region<br>
sizes), if we take circles with different sizes, the segmentation fails.<br>
In addition, as there is no useful information in the intensity of the image,<br>
or its gradient, we choose to perform the spectral clustering on a graph<br>
that is only weakly informed by the gradient. This is close to performing<br>
a Voronoi partition of the graph.<br>
In addition, we use the mask of the objects to restrict the graph to the<br>
outline of the objects. In this example, we are interested in<br>
separating the objects one from the other, and not from the background.<br>


In [ ]:
print(__doc__)

Authors:  Emmanuelle Gouillart <emmanuelle.gouillart@normalesup.org><br>
          Gael Varoquaux <gael.varoquaux@normalesup.org><br>
License: BSD 3 clause

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction import image
from sklearn.cluster import spectral_clustering

In [ ]:
l = 100
x, y = np.indices((l, l))

In [ ]:
center1 = (28, 24)
center2 = (40, 50)
center3 = (67, 58)
center4 = (24, 70)

In [ ]:
radius1, radius2, radius3, radius4 = 16, 14, 15, 14

In [ ]:
circle1 = (x - center1[0]) ** 2 + (y - center1[1]) ** 2 < radius1 ** 2
circle2 = (x - center2[0]) ** 2 + (y - center2[1]) ** 2 < radius2 ** 2
circle3 = (x - center3[0]) ** 2 + (y - center3[1]) ** 2 < radius3 ** 2
circle4 = (x - center4[0]) ** 2 + (y - center4[1]) ** 2 < radius4 ** 2

#############################################################################<br>
4 circles

In [ ]:
img = circle1 + circle2 + circle3 + circle4

We use a mask that limits to the foreground: the problem that we are<br>
interested in here is not separating the objects from the background,<br>
but separating them one from the other.

In [ ]:
mask = img.astype(bool)

In [ ]:
img = img.astype(float)
img += 1 + 0.2 * np.random.randn(*img.shape)

Convert the image into a graph with the value of the gradient on the<br>
edges.

In [ ]:
graph = image.img_to_graph(img, mask=mask)

Take a decreasing function of the gradient: we take it weakly<br>
dependent from the gradient the segmentation is close to a voronoi

In [ ]:
graph.data = np.exp(-graph.data / graph.data.std())

Force the solver to be arpack, since amg is numerically<br>
unstable on this example

In [ ]:
labels = spectral_clustering(graph, n_clusters=4, eigen_solver='arpack')
label_im = np.full(mask.shape, -1.)
label_im[mask] = labels

In [ ]:
plt.matshow(img)
plt.matshow(label_im)

#############################################################################<br>
2 circles

In [ ]:
img = circle1 + circle2
mask = img.astype(bool)
img = img.astype(float)

In [ ]:
img += 1 + 0.2 * np.random.randn(*img.shape)

In [ ]:
graph = image.img_to_graph(img, mask=mask)
graph.data = np.exp(-graph.data / graph.data.std())

In [ ]:
labels = spectral_clustering(graph, n_clusters=2, eigen_solver='arpack')
label_im = np.full(mask.shape, -1.)
label_im[mask] = labels

In [ ]:
plt.matshow(img)
plt.matshow(label_im)

In [ ]:
plt.show()